In [1]:
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import keras
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, population.shape[0]):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 2:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
all_geometry.shape

(38, 8, 3)

In [5]:
def fitness_function(pop,val):
    fitness=[0]*pop.shape[0]

    for i in range(pop.shape[0]):
        
        if (val[i][0][0][0]>=-2 and val[i][0][0][0]<=2) and (val[i][1][0][0]>=-2 and val[i][1][0][0]<=2):
            camber_fitness=abs(val[i][1][0][0]-val[i][0][0][0])/5
            fitness[i]+=camber_fitness
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][1]>=-1 and val[i][0][0][1]<=1) and (val[i][1][0][1]>=-1 and val[i][1][0][1]<=1):
            toe_fitness=abs(val[i][1][0][1]-val[i][0][0][1])/4
            fitness[i]+=toe_fitness
        else:
            fitness[i]=99999
            continue
        
        if (val[i][0][0][2]>=4 and val[i][0][0][2]<=9) and (val[i][1][0][2]>=4 and val[i][1][0][2]<=9):
            fitness[i]+=abs(val[i][1][0][2]-val[i][0][0][2])/2
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][3]>=3 and val[i][0][0][3]<=8) and (val[i][1][0][3]>=3 and val[i][1][0][3]<=8):
            fitness[i]+=abs(val[i][1][0][3]-val[i][0][0][3])/2
        else:
            fitness[i]=99999
            continue

    return fitness

In [6]:
#Use tournament selection

In [7]:
def boltzmann_selection(alpha,current_gen,total_gen,To,fitness,no_parents,pop):
    
    fitness=[1/(x+1e-4) for x in fitness]
    k=1+100*(current_gen/total_gen)
    parents=np.empty([no_parents,pop.shape[1],pop.shape[2]])
    T=To*((1-alpha)**k)
    fmax=max(fitness)
    prob_bs=[0]*pop.shape[0]
    
    for i in range(pop.shape[0]):
        prob_bs[i]=math.exp(-(fmax-fitness[i])/T)
    
    cdf=[prob_bs[0]]
    for j in range(1,pop.shape[0]):
        cdf.append(cdf[-1]+prob_bs[i])
        
    for i in range(1,no_parents):
        selected=random.choices(pop,weights=cdf,k=1)
        parents[i]=selected[0]
        
    return parents

In [8]:
def crossover(parent,no_offspring,alpha):
    
    offspring=np.empty([no_offspring,parent.shape[1],parent.shape[2]])
    
    for i in range(no_offspring):
        parent1_index=i%parent.shape[0]
        parent2_index=(i+1)%parent.shape[0]
        n=random.random()
        offspring[i]=n*parent[parent1_index]+(1-n)*parent[parent2_index]
        
    return offspring

In [9]:
def mutation(offspring,mutation_rate):
    
    mutation=np.array(offspring,copy=True)
        
    for k in range(offspring.shape[0]):
        m=random.random()
        if m>mutation_rate:
            continue
            
        for j in range(offspring.shape[1]):
            for i in range(offspring.shape[2]):
                sign=random.randint(0,1)
                if sign==0:
                    mutation[k][j][i]=mutation[k][j][i]+random.random()
                else:
                    mutation[k][j][i]=mutation[k][j][i]-random.random()

    
    return mutation

In [10]:
def prepare_dataset(pop,model):
    X_flat=pop.reshape((pop.shape[0], -1))
    X=[]
    X_final=[]
    bump_val=[200,-60]
        
    for i in range(X_flat.shape[0]):
        for j in range(len(bump_val)):
            X.append(list(np.append(X_flat[i],bump_val[j])))
        #X_final.append(X)
        #X=[]
            
    X_final=np.array(X)
        
    train_df=pd.DataFrame(X_final)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_train = scaler.fit_transform(train_df)
    scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
        
    val=[]
    val_total=[]
    X = scaled_train_df[scaled_train_df.columns[:25]].values
    count=0
    for i in range(X.shape[0]):
        count+=1
        val1=model.predict(X[i].reshape(1,X[i].shape[0]))
        val.append(val1)
        if count==2:
            val_total.append(val)
            val=[]
            count=0
    return val_total

In [11]:
model=keras.models.load_model('GA_ANN_normal.h5')

In [12]:
def optimization(pop,no_generation,no_parent,mutation_rate):
    
    for m in range(no_generation):

        val_total=prepare_dataset(pop,model)
        pop_size=38
        no_offspring=pop_size-no_parent
        fitness=fitness_function(pop,val_total)
        parent=boltzmann_selection(0.6,m,no_generation,10,fitness,no_parent,pop)
        offspring=crossover(parent,no_offspring,0.7)
        mutate=mutation(offspring,mutation_rate)
        pop=np.concatenate((pop,mutate),axis=0)
        
    return pop,fitness

In [13]:
pop,fitness=optimization(all_geometry,10,20,0.15)

In [14]:
index=fitness.index(min(fitness))

In [15]:
index

0

In [16]:
fitness[index]

99999

In [17]:
pop[index]

array([[3939, -180, 485],
       [4259, -180, 455],
       [4082, -464.5, 255],
       [4072.5, -185, 630],
       [4312, -185, 600],
       [4098.5, -453.5, 420],
       [4031.5, -490.5, 311],
       [4050.5, -208, 522]], dtype=object)

In [19]:
val_t=prepare_dataset(pop,model)

In [25]:
val_t[4]

[array([[-0.37644005,  4.6495376 ,  4.3686037 ,  5.3890967 ]],
       dtype=float32),
 array([[-0.07600115,  2.392653  ,  5.006789  ,  6.292094  ]],
       dtype=float32)]

In [17]:
fitness

[99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,


In [26]:
pop

array([[[3939, -180, 485],
        [4259, -180, 455],
        [4082, -464.5, 255],
        ...,
        [4098.5, -453.5, 420],
        [4031.5, -490.5, 311],
        [4050.5, -208, 522]],

       [[3940, -181, 485],
        [4259, -181, 455],
        [4080, -464.5, 255],
        ...,
        [4098.5, -453.5, 420],
        [4031.5, -490.5, 311],
        [4050.5, -208, 522]],

       [[3945, -181, 485],
        [4266, -181, 460],
        [4084, -464.5, 255],
        ...,
        [4098.5, -453.5, 400],
        [4031.5, -490.5, 315],
        [4050.5, -210, 522]],

       ...,

       [[3941.225668858413, -195.5618663964852, 485.51165560147365],
        [4258.793923126017, -195.60126942235223, 455.87012894912004],
        [4081.7823186131536, -480.34723840283266, 257.87333562323],
        ...,
        [4098.2146128545155, -469.2857033696715, 422.88776276108365],
        [4031.6313434195563, -503.62048966133153, 312.9345901197157],
        [4038.0443371997544, -220.7154420235684, 523.7251423